# 03c - BQML + Vertex AI > Pipelines - автоматизовані конвеєри для оновлення моделей

З часом відбуваються зміни:
- розподіл вхідних даних у наших моделях може зміститися порівняно з тим, коли модель була навчена
- вхідні дані для наших моделей можуть зміщуватися з часом
- можуть стати доступними нові вхідні дані/особливості
- може бути створена краща модель

У цьому зошиті ми побудуємо модель-суперника з тими ж навчальними даними, також використовуючи BQML, але з іншим типом моделі - глибокою нейронною мережею.  Ми побудуємо конвеєр Vertex AI Pipeline для організації процесу побудови нової моделі, порівняння з розгорнутою моделлю та умовної заміни розгорнутої моделі на нову.

Цей процес може бути запущений на основі часу, що минув, кількості нових даних, виявленого перекосу в навчанні або навіть дрейфу прогнозу за допомогою Vertex AI Monitoring.  

### Огляд:
- Створення власних компонентів конвеєра
    - Використовуйте BigQuery ML для отримання прогнозів та Scikit-Learn для розрахунку метрик моделі
    - Використовуйте BigQuery ML для навчання нової моделі - глибокої нейронної мережі
    - Порівняйте метрики моделі для базової моделі та моделі-конкурента
    - Експортуйте модель BigQuery ML до Google Cloud Storage
    - Замінити модель, розгорнуту на кінцевій точці, на модель-конкурента, деінсталювати попередню модель
- Визначення конвеєрного потоку
- Скомпілюйте конвеєр
- Запустіть конвеєр у Vertex AI
- Отримайте прогнози з оновленої кінцевої точки


---
## Налаштування

Вхідні дані:

In [ ]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

In [2]:
REGION = 'us-central1'
DATANAME = 'fraud'
NOTEBOOK = '03c'

# ресурси
DEPLOY_IMAGE='us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-3:latest'
DEPLOY_COMPUTE = 'n1-standard-4'

# модель
VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id'

пакети:

In [3]:
from google.cloud import aiplatform
from datetime import datetime
from typing import NamedTuple
import kfp # used for dsl.pipeline
import kfp.v2.dsl as dsl # used for dsl.component, dsl.Output, dsl.Input, dsl.Artifact, dsl.Model, ...
# from google_cloud_pipeline_components import aiplatform as gcc_aip

from google.cloud import bigquery
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
import json
import numpy as np

клієнти:

In [4]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bq = bigquery.Client()

параметри:

In [5]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET = PROJECT_ID
URI = f"gs://{BUCKET}/{DATANAME}/models/{NOTEBOOK}"
DIR = f"temp/{NOTEBOOK}"

In [ ]:
# Надайте обліковому запису служби roles/storage.objectAdmin права доступу
# Консоль > IMA > Вибрати обліковий запис <номер проекту>-compute@developer.gserviceaccount.com > редагувати - надати роль
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

оточення:

In [7]:
!rm -rf {DIR}
!mkdir -p {DIR}

---
## Користувацькі компоненти (KFP)

Конвеєри Vertex AI складаються з компонентів, які працюють незалежно, з входами і виходами, які з'єднуються у граф - конвеєр.  Для цього робочого процесу в блокноті використовуються наступні користувацькі компоненти для організації навчання моделі-кандидата, оцінки моделі-кандидата та існуючої моделі, порівняння їх на основі метрик моделі, якщо модель-кандидат краща, то замінити модель, яка вже розгорнута на існуючій кінцевій точці.  Ці кастомні компоненти побудовані як функції python!

### Метрики моделі
- Отримання прогнозів для тестових даних з моделі BigQuery
- Обчисліть [average_precision_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html#sklearn.metrics.average_precision_score)

In [36]:
@dsl.component(
    base_image = 'python:3.10',
    packages_to_install = ['pandas','db-dtypes','pyarrow','scikit-learn','google-cloud-bigquery']
)
def bqml_eval(
    project: str,
    var_target: str,
    model: str,
    dataname: str,
    metrics: dsl.Output[dsl.Metrics],
    metricsc: dsl.Output[dsl.ClassificationMetrics]
) -> NamedTuple("model_eval", [("metric", float)]):

    from collections import namedtuple
    from sklearn.metrics import average_precision_score, confusion_matrix
    from google.cloud import bigquery
    bq = bigquery.Client(project = project)

    query = f"""
    SELECT {var_target}, predicted_{var_target}, prob, splits 
    FROM ML.PREDICT (MODEL `{project}.{dataname}.{model}`,(
        SELECT *
        FROM `{project}.{dataname}.{dataname}_prepped`
        WHERE splits = 'TEST')
      ), UNNEST(predicted_{var_target}_probs)
    WHERE label=1
    """
    pred = bq.query(query = query).to_dataframe()

    auPRC = average_precision_score(pred[var_target].astype(int), pred['prob'], average='micro')    
    metrics.log_metric('auPRC', auPRC)
    metricsc.log_confusion_matrix(['Not Fraud', 'Fraud'], confusion_matrix(pred[var_target].astype(int), pred[f'predicted_{var_target}'].astype(int)).tolist())
    
    model_eval = namedtuple("model_eval", ["metric"])
    return model_eval(metric = float(auPRC))

### BigQuery - Навчання DNN

In [37]:
@dsl.component(
    base_image = 'python:3.9',
    packages_to_install = ['google-cloud-bigquery']
)
def bqml_dnn(
    project: str,
    var_target: str,
    var_omit: str,
    model: str,
    dataname: str,
    bqml_model: dsl.Output[dsl.Artifact]
) -> NamedTuple("bqml_training", [("query", str)]):
    
    from collections import namedtuple
    from google.cloud import bigquery
    bq = bigquery.Client(project = project)
    
    query = f"""
    CREATE OR REPLACE MODEL `{project}.{dataname}.{model}`
    OPTIONS
        (model_type = 'DNN_CLASSIFIER',
            auto_class_weights = FALSE,
            input_label_cols = ['{var_target}'],
            data_split_col = 'custom_splits',
            data_split_method = 'CUSTOM',
            EARLY_STOP = FALSE,
            OPTIMIZER = 'SGD',
            HIDDEN_UNITS = [2],
            LEARN_RATE = 0.001,
            ACTIVATION_FN = 'SIGMOID',
            MAX_ITERATIONS = 10,
            HPARAM_TUNING_ALGORITHM = 'VIZIER_DEFAULT',
            HPARAM_TUNING_OBJECTIVES = ['ROC_AUC'],
            DROPOUT = HPARAM_RANGE(0, 0.8),
            BATCH_SIZE = HPARAM_RANGE(8, 500),
            MAX_PARALLEL_TRIALS = 2,
            NUM_TRIALS = 20
        ) AS
    SELECT * EXCEPT({','.join(var_omit.split())}, splits),
        CASE
            WHEN splits = 'VALIDATE' THEN 'EVAL'
            ELSE splits
        END AS custom_splits
    FROM `{project}.{dataname}.{dataname}_prepped`
    WHERE splits != 'TEST'
    """
    job = bq.query(query = query)
    job.result()
    bqml_model.uri = f"bq://{project}.{dataname}.{model}"
    
    result = namedtuple("bqml_training", ["query"])
                
    return result(query = str(query))

### Порівняння моделей

In [38]:
@dsl.component
def model_compare(
    base_metric: float,
    challenger_metric: float,
) -> bool: 
    
    if base_metric < challenger_metric:
        replace = True
    else:
        replace = False
    
    return replace

### Експорт моделі BQML

In [39]:
@dsl.component(
    base_image = 'python:3.9',
    packages_to_install = ['google-cloud-bigquery']
)
def bqml_export(
    project: str,
    export_location: str,
    bqml_model: dsl.Input[dsl.Model],
    tf_model: dsl.Output[dsl.Artifact],  
):
    
    from google.cloud import bigquery
    bq = bigquery.Client(project = project)
    
    bqml_model_name = bqml_model.uri.split("/")[-1]
    export = bq.query(query = f"EXPORT MODEL `{bqml_model_name}` OPTIONS(URI = '{export_location}')")
    export.result()
    
    tf_model.uri = export_location

### Замінити модель на кінцевій точці

In [40]:
@dsl.component(
    packages_to_install = ['google-cloud-aiplatform'],
    base_image = 'python:3.9'
)
def endpoint_update(
    project: str,
    region: str,
    endpoint_prefix: str,
    newmodel: dsl.Input[dsl.Model],
    display_name: str,
    deploy_machine: str,
    deploy_container: str,
    label: str
):
    
    from google.cloud import aiplatform
    aiplatform.init(project = project, location = region)

    # завантажити нову модель
    model = aiplatform.Model.upload(
        display_name = display_name,
        serving_container_image_uri = deploy_container,
        artifact_uri = newmodel.uri,
        labels = {'notebook':f'{label}'}
    )
    
    # знайти кінцеву точку
    for e in aiplatform.Endpoint.list():
        if e.display_name.startswith(endpoint_prefix): endpoint = e
    print(endpoint.display_name)

    # перелічити моделі на кінцевій точці
    models = endpoint.list_models()
    if len(models) == 1:
        oldmodel = models[0]
    print(oldmodel)
    
    # розгорнути модель на кінцевій точці з traffic_split = 100
    endpoint.deploy(
        model = model,
        deployed_model_display_name = display_name,
        traffic_percentage = 100,
        machine_type = deploy_machine,
        min_replica_count = 1,
        max_replica_count = 1
    )
    
    # згорнути модель
    endpoint.undeploy(
        deployed_model_id = oldmodel.id
    )

---
## Ініціалізація Pipeline (KFP)

In [41]:
@dsl.pipeline(
    name = f'kfp-{NOTEBOOK}-{DATANAME}-{TIMESTAMP}', 
    pipeline_root = URI+'/'+str(TIMESTAMP)+'/kfp/'
)
def pipeline(
    endpoint_prefix: str,
    project: str = PROJECT_ID,
    region: str = REGION,
    dataname: str = DATANAME,
    display_name: str = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    deploy_machine: str = DEPLOY_COMPUTE,
    deploy_container: str = DEPLOY_IMAGE,
    bq_source: str = f'bq://{PROJECT_ID}.{DATANAME}.{DATANAME}_prepped',
    var_target: str = VAR_TARGET,
    var_omit: str = VAR_OMIT,
    uri: str = URI,
    label: str = NOTEBOOK 
):
        
    # отримати AUC для поточної моделі
    base_model_eval = bqml_eval(
        project = project,
        var_target = var_target,
        model = f'{dataname}_lr',
        dataname = dataname
    )
    base_model_eval.set_caching_options(False)
    
    # навчання моделі-претендента
    challenger_model = bqml_dnn(
        project = project,
        var_target = var_target,
        var_omit = var_omit,
        model = f'{dataname}_dnn',
        dataname = dataname,
    )
    challenger_model.set_caching_options(True)
    
    # AUC моделі претендента
    challenger_model_eval = bqml_eval(
        project = project,
        var_target = var_target,
        model = f'{dataname}_dnn',
        dataname = dataname
    )
    challenger_model_eval.set_caching_options(False)
    challenger_model_eval.after(challenger_model)
    
    # порівняння моделей
    compare = model_compare(
        base_metric = base_model_eval.outputs["metric"],
        challenger_metric = challenger_model_eval.outputs["metric"]
    )
    
    # умови розгортання
    with dsl.Condition(
        compare.output == 'true',
        name = "replace_model"
    ):
        # експорт моделі BQML
        export = bqml_export(
            project = project,
            export_location = uri,
            bqml_model = challenger_model.outputs["bqml_model"]
        )
        
        # заміна моделі в кінцевій точці
        replace = endpoint_update(
            project = project,
            region = region,
            endpoint_prefix = endpoint_prefix,
            newmodel = export.outputs["tf_model"],
            display_name = display_name,
            deploy_machine = deploy_machine,
            deploy_container = deploy_container,
            label = label
        )

---
## Компіляція Pipeline

In [ ]:
kfp.v2.compiler.Compiler().compile(
    pipeline_func = pipeline,
    package_path = f"{DIR}/{NOTEBOOK}.json"
)

Перемістіть скомпільовані файли конвеєра до GCS Bucket

In [ ]:
!gsutil cp {DIR}/{NOTEBOOK}.json {URI}/{TIMESTAMP}/kfp/

---
## Створення Vertex AI Pipeline Job

In [44]:
pipeline = aiplatform.PipelineJob(
    display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    template_path = f"{URI}/{TIMESTAMP}/kfp/{NOTEBOOK}.json",
    pipeline_root = f"{URI}/{TIMESTAMP}/kfp/",
    parameter_values = {"endpoint_prefix": "03b"},
    enable_caching = None,
    labels = {'notebook':f'{NOTEBOOK}'}
)

In [ ]:
response = pipeline.run(service_account = SERVICE_ACCOUNT)

In [46]:
aiplatform.get_pipeline_df(pipeline = f'kfp-{NOTEBOOK}-{DATANAME}-{TIMESTAMP}')

pipeline_name                                     run_name  \
0  kfp-03c-fraud-20220707160930  kfp-03c-fraud-20220707160930-20220707162146   
1  kfp-03c-fraud-20220707160930  kfp-03c-fraud-20220707160930-20220707161243   

  param.input:var_omit                           param.input:bq_source  \
0       transaction_id  bq://statmike-mlops-349915.fraud.fraud_prepped   
1       transaction_id  bq://statmike-mlops-349915.fraud.fraud_prepped   

  param.input:label  param.input:display_name param.input:region  \
0               03c  03c_fraud_20220707160930        us-central1   
1               03c  03c_fraud_20220707160930        us-central1   

  param.input:dataname                       param.input:deploy_container  \
0                fraud  us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu...   
1                fraud  us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu...   

     param.input:project                              param.input:uri  \
0  statmike-mlops-349915  gs://statmike-mlops-349915/fraud/models/03c   
1  statmike-mlops-349915  gs://statmike-mlops-349915/fraud/models/03c   

  param.input:endpoint_prefix param.input:deploy_machine  \
0                         03b              n1-standard-4   
1                         03b              n1-standard-4   

  param.input:var_target                             metric.confusionMatrix  \
0                  Class  {'annotationSpecs': [{'displayName': 'Not Frau...   
1                  Class                                                NaN   

   metric.auPRC  
0      0.806335  
1           NaN

---
## Прогнозування

### Спостереження для прогнозування

In [47]:
pred = bq.query(query = f"SELECT * FROM {DATANAME}.{DATANAME}_prepped WHERE splits='TEST' LIMIT 10").to_dataframe()

In [48]:
pred.head(4)

Time        V1        V2        V3        V4        V5        V6  \
0   32799  1.153477 -0.047859  1.358363  1.480620 -1.222598 -0.481690   
1   35483  1.286640  0.072917  0.212182 -0.269732 -0.283961 -0.663306   
2  163935  1.961967 -0.247295 -1.751841 -0.268689  0.956431  0.707211   
3   30707 -0.964364  0.176372  2.464128  2.672539  0.145676 -0.152913   

         V7        V8        V9  ...       V23       V24       V25       V26  \
0 -0.654461  0.128115  0.907095  ... -0.025964  0.701843  0.417245 -0.257691   
1 -0.016385 -0.120297 -0.135962  ...  0.052674  0.076792  0.209208  0.847617   
2  0.020675  0.189433  0.455055  ...  0.186420 -1.621368 -0.131098  0.034276   
3 -0.591983  0.305066 -0.148034  ... -0.024200  0.365226 -0.745369 -0.060544   

        V27       V28  Amount  Class                        transaction_id  \
0  0.060115  0.035332     0.0      0  e9d16028-4b41-4753-87ee-041d33642ae9   
1 -0.086559 -0.008262     0.0      0  8b319d3a-2b2d-445b-a9a2-0da3d664ec2a   
2 -0.004909 -0.090859     0.0      0  788afb87-60aa-4482-8b48-c924bec634aa   
3  0.095692  0.217639     0.0      0  473d0936-1974-4ae8-ab70-230e7599bd3f   

   splits  
0    TEST  
1    TEST  
2    TEST  
3    TEST  

[4 rows x 33 columns]

In [49]:
newob = pred[pred.columns[~pred.columns.isin(VAR_OMIT.split()+[VAR_TARGET, 'splits'])]].to_dict(orient='records')[0]
newob

{'Time': 32799,
 'V1': 1.15347743766561,
 'V2': -0.0478588055804616,
 'V3': 1.35836288729212,
 'V4': 1.48062009487976,
 'V5': -1.22259808550513,
 'V6': -0.481689608379461,
 'V7': -0.6544612667240159,
 'V8': 0.128114599402494,
 'V9': 0.907094671648477,
 'V10': -0.0364175882073298,
 'V11': -0.659895142180526,
 'V12': -0.307334930019039,
 'V13': -1.3656343720183501,
 'V14': 0.035305634441467997,
 'V15': 0.7114399756440071,
 'V16': 0.25384340384905,
 'V17': -0.22283165732475999,
 'V18': 0.26559908021792394,
 'V19': -0.5970199308963089,
 'V20': -0.24568562944008399,
 'V21': 0.12551444308840598,
 'V22': 0.480049308608633,
 'V23': -0.0259637518455547,
 'V24': 0.7018428515999721,
 'V25': 0.41724451503333504,
 'V26': -0.257690694038964,
 'V27': 0.06011458360133701,
 'V28': 0.0353320583585812,
 'Amount': 0.0}

### Отримання прогнозу: Python Client

In [50]:
for e in aiplatform.Endpoint.list():
    if e.display_name.startswith('03b'): endpoint = e
print(endpoint.display_name)
print(endpoint.resource_name)

03b_fraud_20220707104636
projects/1026793852137/locations/us-central1/endpoints/6922833071733473280


In [51]:
client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
predictor = aiplatform.gapic.PredictionServiceClient(client_options = client_options)

In [52]:
from google.api import httpbody_pb2
instances = {"instances": [newob], "signature_name": "predict"}
http_body = httpbody_pb2.HttpBody(data = json.dumps(instances).encode("utf-8"), content_type = "application/json")

In [53]:
pred = predictor.raw_predict(endpoint = endpoint.resource_name, http_body = http_body)

In [54]:
json.loads(pred.data)

{'predictions': [{'classes': ['0'],
   'all_class_ids': [0, 1],
   'logistic': [0.999906898],
   'all_classes': ['1', '0'],
   'probabilities': [9.31419345e-05, 0.999906898],
   'logits': [9.28129292],
   'class_ids': [1]}]}

### Отримання прогнозу: REST

In [55]:
with open(f'{DIR}/request.json','w') as file:
    file.write(json.dumps({"signature_name": "predict", "instances": [newob]}))

In [56]:
!curl -X POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @{DIR}/request.json \
https://{REGION}-aiplatform.googleapis.com/v1/{endpoint.resource_name}:rawPredict

{
    "predictions": [
        {
            "probabilities": [9.31419345e-05, 0.999906898],
            "logits": [9.28129292],
            "class_ids": [1],
            "classes": ["0"],
            "all_class_ids": [0, 1],
            "logistic": [0.999906898],
            "all_classes": ["1", "0"]
        }
    ]
}

### Отримання прогнозу: gcloud (CLI)

In [57]:
with open(f'{DIR}/request.json','w') as file:
    file.write(json.dumps({"signature_name": "predict", "instances": [newob]}))

In [58]:
!gcloud ai endpoints raw-predict {endpoint.name.rsplit('/',1)[-1]} --region={REGION} --request=@{DIR}/request.json

Using endpoint [https://us-central1-aiplatform.googleapis.com/]
{
    "predictions": [
        {
            "logistic": [0.999906898],
            "all_classes": ["1", "0"],
            "probabilities": [9.31419345e-05, 0.999906898],
            "logits": [9.28129292],
            "class_ids": [1],
            "classes": ["0"],
            "all_class_ids": [0, 1]
        }
    ]
}